# Energy-based Optimization

This notebook contains the code for energy-based scheduling. First, the synthetically generated input data is read in. This is followed by the modeling of the optimization problem in Gurobi. At the end, an output with the optimal scheduling times is generated. 

### Import Gurobi 

In [1]:
import gurobipy as gp
from gurobipy import GRB

### 1) Data PreProcessing

In [11]:
import pandas as pd 
import datetime

# read data 
df = pd.read_csv('sensor_neu.csv')
df.drop(columns=df.columns[0], axis=1, inplace=True) # drop index column 
df['dateTime'] = pd.to_datetime(df.dateTime) # parse timestamps
df = df.rename(columns={'output (kWh)': 'output', 'basicConsumption (kWh)': 'basic', 'managementConsumption (kWh)': 'management', 'productionConsumption (kWh)': 'production'}) # rename columns

# select planing period of 2 weeks, starting with monday 
df = df[(df['dateTime'] >= '2022-07-04 00:00:00') & (df['dateTime'] <= '2022-07-17 23:00:00')]

# calculate netzbezug
df['balance'] = (df['basic'] + df['management'] + df['production']) - df['output']
df = df.drop(['basic', 'management', 'production', 'output'], axis=1)

# reformat day and hour 
df['hour'] = df['dateTime'].dt.hour
n=24
df['day'] = [int(i/n) for i,x in enumerate(df.index)]
netzbezug = df

# set planning times of 2 weeks 
days = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
hours = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
                          
# read appointment data
termine_df = pd.read_csv('termine.csv', sep=";")
termine_energy = dict(termine_df[['Termin','Energieverbrauch']].values) 
termine_length = dict(termine_df[['Termin','Dauer']].values)
print(termine_df)

   Termin Terminbezeichnung  Energieverbrauch  Dauer
0       0           Termin1                80      4
1       1           Termin2               120      6
2       2           Termin3               100      5


### 2) Create new model 

In [3]:
model = gp.Model("energy based scheduling")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-31


### 3) Create variables

In [4]:
# energy consumption per appointment
consumption = model.addVars(days,hours,termine_energy,vtype=GRB.CONTINUOUS,name="consumption")

# planned start of appointment 
start = model.addVars(consumption, vtype=GRB.BINARY, name="start")
end = model.addVars(consumption, vtype=GRB.BINARY, name="end")

# save start day und hour as numerical value
start_hour = model.addVars(termine_energy,vtype=GRB.CONTINUOUS,name="start_hour")
start_day = model.addVars(termine_energy,vtype=GRB.CONTINUOUS,name="start_day")

# save end hour as numerical value 
end_hour = model.addVars(termine_energy,vtype=GRB.CONTINUOUS,name="end_hour")

### 4) Calculate Netzbezug while appointment

In [5]:
# calculate netzbezug of appointment
for termin in termine_energy:
    for day in days: 
        for hour in range(0,18):
            for i in range(0,termine_length[termin]):
                consumption[day,hour,termin] = consumption[day,hour,termin]+netzbezug['balance'][(netzbezug['day'] == day) & (netzbezug['hour'] == hour+i)]+(termine_energy[termin]/termine_length[termin])              

### 5) Set objective function

In [6]:
# minimize netzbezug
obj = gp.quicksum((consumption[day,hour,termin]*start[day,hour,termin])
                 for day in days for hour in hours for termin in termine_energy)

model.setObjective(obj, GRB.MINIMIZE)

### 6) Add constraints 

In [7]:
# weekend constraint
for termin in termine_energy:
    for hour in hours:
        for day in days:
            if day in [5,6,12,13]: 
                model.addConstr((start[day,hour,termin])==0)
                
# only 1 start time per appointment
for termin in termine_energy: 
    model.addConstr(gp.quicksum(start[day,hour,termin] 
                    for day in days for hour in hours) == 1)

# no overlap constraint
for day in days: 
    for hour in hours:
        if hour < 18:
            model.addConstr((start[day,hour,0] == 1) >> (gp.quicksum(start[day,hour+i,1] 
                                                                 for i in range(1,termine_length[0]))==0))
            model.addConstr((start[day,hour,1] == 1) >> (gp.quicksum(start[day,hour+i,0] 
                                                                 for i in range(1,termine_length[1]))==0))
            model.addConstr((start[day,hour,0] == 1) >> (gp.quicksum(start[day,hour+i,2] 
                                                                 for i in range(1,termine_length[0]))==0))
            model.addConstr((start[day,hour,2] == 1) >> (gp.quicksum(start[day,hour+i,0] 
                                                                 for i in range(1,termine_length[2]))==0))
            model.addConstr((start[day,hour,1] == 1) >> (gp.quicksum(start[day,hour+i,2] 
                                                                 for i in range(1,termine_length[1]))==0))
            model.addConstr((start[day,hour,2] == 1) >> (gp.quicksum(start[day,hour+i,1] 
                                                                 for i in range(1,termine_length[2]))==0))

# no overlap of start times 
for day in days:
    for hour in hours:
        model.addConstr(start[day,hour,0]+start[day,hour,1]+start[day,hour,2]<=1)
            
# save start hour and day of appointment 
for termin in termine_energy: 
    for day in days: 
        for hour in hours:
            model.addConstr((start[day,hour,termin]==1) >> (start_day[termin]==day))
            model.addConstr((start[day,hour,termin]==1) >> (start_hour[termin]==hour))

# set end time of appointment 
for termin in termine_length:            
    model.addConstr(end_hour[termin]==start_hour[termin]+termine_length[termin])      
    
# end time constraint
for termin in termine_length:            
    model.addConstr(end_hour[termin] <= 18)      
    
# start time constraint 
for termin in termine_length:            
    model.addConstr(start_hour[termin] >= 8)      

### 7) Optimize model

In [8]:
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 636 rows, 3033 columns and 2316 nonzeros
Model fingerprint: 0xff86160e
Model has 1008 quadratic objective terms
Model has 3528 general constraints
Variable types: 1017 continuous, 2016 integer (2016 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
  GenCon rhs range [1e+00, 2e+01]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 401 rows and 2688 columns
Presolve time: 0.03s
Presolved: 235 rows, 345 columns, 1444 nonzeros
Presolved model has 180 SOS constraint(s)
Variable types: 165 continuous, 180 integer (180 binary)
Found heuristic solution: objective 123.8452000

Root relaxation: objective 6.946000e+00, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Curre

### 8) Generate output 

In [12]:
# save planned appointments
appointments = pd.DataFrame(columns=['Termin', 'Start_Day', 'Start_Hour'])
for v in model.getVars():
    if v.VarName.startswith("start_day"): 
        appointments = appointments.append({'Termin':v.VarName, 'Start_Day':int(v.X)}, ignore_index=True)                
    if v.VarName.startswith("start_hour"):
        appointments = appointments.append({'Termin':v.VarName, 'Start_Hour':int(v.X)}, ignore_index=True)

# reformat dataframe
appointments['Termin'] = appointments['Termin'].map(lambda x: x.lstrip('start_hourday[').rstrip(']'))
appointments = appointments.groupby(by="Termin").sum().reset_index()

# merge with dataframe with timestamps
start_times = pd.merge(netzbezug, appointments,  how='left', left_on=['hour','day'], right_on = ['Start_Hour','Start_Day']).dropna()
start_times = start_times.drop(['balance', 'Start_Day', 'Start_Hour', 'hour', 'day'], axis=1)
start_times = start_times.rename(columns={'dateTime': 'Start_DateTime'}) # rename column
start_times.to_csv('optimized.csv')

print(start_times)

        Start_DateTime Termin
12 2022-07-04 12:00:00      1
61 2022-07-06 13:00:00      0
84 2022-07-07 12:00:00      2


<ipython-input-12-01527c4108e6>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appointments = appointments.append({'Termin':v.VarName, 'Start_Hour':int(v.X)}, ignore_index=True)
<ipython-input-12-01527c4108e6>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appointments = appointments.append({'Termin':v.VarName, 'Start_Hour':int(v.X)}, ignore_index=True)
<ipython-input-12-01527c4108e6>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appointments = appointments.append({'Termin':v.VarName, 'Start_Hour':int(v.X)}, ignore_index=True)
<ipython-input-12-01527c4108e6>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appointments = appointm